In [ ]:
!pip install torch spacy umap-learn pandas altair sentence-transformers scipy beautifulsoup4
!python -m spacy download en_core_web_sm
!python -m spacy download nb_core_news_sm
!python -c "from sentence_transformers import SentenceTransformer; SentenceTransformer('distiluse-base-multilingual-cased-v2');"

# 📝 🧮 💡 **Text to numbers to insights**

Coming up in this talk:

- A high level intro on neural NLP: How do we get from text to numbers?
- Some practical clicking around this notebook to see what it means in practice

**Not** coming up in this talk:

- Generative LLMs
- ChatGPT
- AI Hype

We are going to move quickly, do get back to this notebook on your own time or hmu on email or LinkedIn if you want to go deeper ✨

#### 👨🏻 **About me**

- ~8 years in Academia, PhD in NLP 2019
- Worked in a couple ML startups
- Team lead MI@Fremtind

---


### 🧐 **Language is hard**

> _*yo no soy marinero, soy capitan*_

There are many languages

> _*that jaguar is sick*_

Language is contextual

> _*a boujee elon stan*_

Language is constantly evolving

#### **How do we represent language?**

- Tabular data is collected facts
- Pictures are RGB matrices
- Words (and meaningful sequences of words) have been problematic

---

### 🎓 **Distributional semantics**

> _You shall know a word by the company it keeps_ (Firth, 1957)

```
My son loves to eat [bananas].
[Cookies] are sweet.
[Apples] grow on trees.
I love dunking [biscuits] in milk.
```

- The meaning of a word is a function of the meaning of the words it co-occurs with 🤯

But how do we express that function? What is its output?
→ We need some sort of universal function approximator

---

### 🔮 **Neural representations**

**Word2Vec**, The first successfull neural language representation, extremely high level:

- Get a lot of data
- Split all the words
- Train a feed-forward network to predict a vector of a word given the vectors of the surrounding words
- Update the word vectors by backpropagating the error

→ You end up with a dense representation for each word. This representation has semantic properties!

```python
distance(vectors["banana"], vectors["cake"]) < distance(vectors["banana"], vectors["lego"])

vectors.most_similar(vectors["apple"] - vectors["fruit"] + vectors["potato"])
=> "vegetable"
```

#### **Nowadays**

- Contextual embeddings
- Longer text representations, eg sentences, documents
- Transformer architectures

---

In [ ]:
import altair as alt
import pandas as pd
import spacy
import torch

from bs4 import BeautifulSoup
from urllib.request import urlopen
from sentence_transformers import SentenceTransformer
from umap import UMAP


PREPROCESS = spacy.load("en_core_web_sm") 
TRANSFORMER = SentenceTransformer("distiluse-base-multilingual-cased-v2")
DEVICE = "mps" # "cpu" or "cuda"


def get_text(url):
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    return " ".join(soup.get_text().split())


def get_data_df(data, name=pd.NA, limit=80):
    data = PREPROCESS(data)
    df = pd.DataFrame()
    df["text"] = [sentence.text for sentence in data.sents if len(sentence.text) > limit]
    df["embedding"] = list(TRANSFORMER.encode(list(df["text"]), device=torch.device(DEVICE)))
    df["name"] = [name for _ in range(len(df))]
    return df


def reduce_dimensions(df):
    reduced = UMAP(random_state=42).fit_transform(list(df["embedding"]))
    df["x"], df["y"] = reduced[:, 0], reduced[:, 1]
    return df

In [ ]:
words = [
    ("banana", "banana", 0),
    ("apple", "apple", 0), 
    ("pineapple", "pineapple", 0), 
    ("orange", "orange", 0),
    ("pear", "pear", 0),
    ("grape", "grape", 0),
    ("strawberry", "strawberry", 0),
    ("work", "work", 0),
    ("school", "school", 0),
    ("university", "university", 0),
    ("college", "college", 0),
    ("student", "student", 0),
]
words = reduce_dimensions(pd.concat([get_data_df(*x) for x in words]))
words


In [ ]:
alt.Chart(words).mark_circle(size=100).encode(
    x="x",
    y="y",
    tooltip=["text", "name"],
    color="name"
).properties(width=500, height=300).interactive()

---

### 🔵 🔴 🟢 **Representing wikipedia articles with a Transformer** 

In [ ]:
blading = get_data_df(get_text("https://en.wikipedia.org/wiki/Inline_skating"), "blading")
muppets = get_data_df(get_text("https://en.wikipedia.org/wiki/The_Muppets"), "muppets")
skateboarding = get_data_df(get_text("https://en.wikipedia.org/wiki/Skateboarding"), "skateboarding")
smurfs = get_data_df(get_text("https://en.wikipedia.org/wiki/The_Smurfs"), "smurfs")
tennis = get_data_df(get_text("https://en.wikipedia.org/wiki/Tennis"), "tennis")
seinfeld = get_data_df(get_text("https://en.wikipedia.org/wiki/Seinfeld"), "seinfeld")

df = pd.concat([blading, muppets, skateboarding, smurfs, tennis, seinfeld])
df = reduce_dimensions(df)

In [ ]:
alt.Chart(df.drop_duplicates(subset=["text"])).mark_circle(size=100).encode(
    x="x",
    y="y",
    tooltip=["text", "name"],
    color="name"
).properties(width=1500, height=800).interactive()

---

### 🔮 **Semantic search on the cheap**

In [ ]:
from scipy.spatial.distance import cosine
from pprint import pprint

query = "How does the smurf language work?"
embedded_query = TRANSFORMER.encode([query])[0]
df["distance"] = [cosine(embedded_query, embedding) for embedding in df["embedding"]]

pprint(df.sort_values(by="distance")["text"].head(3).tolist())

---

### 🔮 **Classification with no training**

In [ ]:
df["label"] = [
    "sports" if x in ("blading", "skateboarding", "tennis") else "tv-shows" 
    for x in df["name"]
]

wiki = get_text("https://en.wikipedia.org/wiki/Futurama")
embedded_wiki = TRANSFORMER.encode([wiki])[0]
df["distance"] = [cosine(embedded_wiki, embedding) for embedding in df["embedding"]]

df.sort_values(by="distance")["label"].head(10).value_counts().idxmax()